# 볼린저 밴드 투자전략

1. 파일 로드
2. 결측치와 이상치를 확인 및 제거
3. 시간과 종가만의 데이터를 가지고 데이터프레임 생성
4. 이동 평균선 : 데이터 20개의 평균을 구해서 새로운 파생변수 생성
5. 상단 밴드 : 이동 평균선 +(2 * 데이터 20개의 표준편차)
6. 하단 밴드 : 이동 평균선 -(2 * 데이터 20개의 표준편차)
7. 구매 상태를 확인하는 파생변수 생성
8. 구매 상태를 입력
9. 수익률 계산

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
# 1. 파일 로드
df = pd.read_csv("../csv/AMZN.csv", index_col = "Date")
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


In [10]:
# 결측치와 이상치를 확인
# np.nan은 결측치, +-np.inf는 이상치
# any에 들어가는 값이 0, 1 이냐에 따라 column 기준이냐 index 기준이냐 차이.
# df[]는 안의 기준으로 필터의 역할을 함.
df[df.isin([np.nan, np.inf, -np.inf]).any(1)]

C:\Users\Kim Taemin\AppData\Local\Temp\ipykernel_24480\1180290720.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df[df.isin([np.nan, np.inf, -np.inf]).any(1)]


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [24]:
# 2. 결측치와 이상치를 제외한 데이터프레임을 생성
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

C:\Users\Kim Taemin\AppData\Local\Temp\ipykernel_24480\4235976942.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]


In [25]:
# 3. 시간과 종가로 데이터프레임 생성
# case 1
df[["Adj Close"]]

# case 2
price_df = df.loc[:, ["Adj Close"]]

In [26]:
# 4. 이동 평균선
# 데이터 20개의 평균의 값
# 현재 날짜에서 과거 20일치의 데이터의 평균
# rolling(N)은 N개씩 묶어주는 역할

price_df["Center"] = price_df["Adj Close"].rolling(20).mean()

In [22]:
price_df.iloc[18:25]

,Adj Close,Center
Date,,
1997-06-11,1.541667,NaN
1997-06-12,1.604167,1.574740
1997-06-13,1.583333,1.555990
1997-06-16,1.572917,1.548177
1997-06-17,1.505208,1.538021
1997-06-18,1.510417,1.531771
1997-06-19,1.510417,1.535938


In [27]:
# 5. 상단 밴드 생성
# Ub : 이동평균선 + (2 * 종가 데이터 20개의 표준편차(std()))
price_df["Ub"] = price_df["Center"] + 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

,Adj Close,Center,Ub
Date,,,
1997-06-11,1.541667,NaN,NaN
1997-06-12,1.604167,1.574740,1.836333
1997-06-13,1.583333,1.555990,1.745696
1997-06-16,1.572917,1.548177,1.719869
1997-06-17,1.505208,1.538021,1.693045
1997-06-18,1.510417,1.531771,1.680201
1997-06-19,1.510417,1.535938,1.676462


In [28]:
# 6. 하단 밴드 생성
# Lb : 이동평균선 - (2 * 종가 데이터 20개의 표준편차(std()))
price_df["Lb"] = price_df["Center"] - 2 * price_df["Adj Close"].rolling(20).std()
price_df.iloc[18:25]

,Adj Close,Center,Ub,Lb
Date,,,,
1997-06-11,1.541667,NaN,NaN,NaN
1997-06-12,1.604167,1.574740,1.836333,1.313146
1997-06-13,1.583333,1.555990,1.745696,1.366283
1997-06-16,1.572917,1.548177,1.719869,1.376485
1997-06-17,1.505208,1.538021,1.693045,1.382996
1997-06-18,1.510417,1.531771,1.680201,1.383341
1997-06-19,1.510417,1.535938,1.676462,1.395413


In [ ]:
price_df.plot()

In [32]:
# 시작 시간 생성
start_time = "2010-01-02"
price_df2 = price_df.loc[start_time :]

In [33]:
# 7. 거래 내역 파생변수 생성
# 비어있는 값
price_df2["Trade"] = ""
price_df2.head()

C:\Users\Kim Taemin\AppData\Local\Temp\ipykernel_24480\1584944405.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df2["Trade"] = ""


,Adj Close,Center,Ub,Lb,Trade
Date,,,,,
2010-01-04,133.899994,133.984001,141.460445,126.507556,
2010-01-05,134.690002,133.839500,141.132776,126.546225,
2010-01-06,132.250000,133.741500,141.066419,126.416581,
2010-01-07,130.000000,133.536000,141.045671,126.026329,
2010-01-08,133.520004,133.646500,141.082939,126.210062,


## 구매내역 추가

- 조건
    1. 상단 밴드보다 종가가 높은 경우
        - 현재 구매 상태인 경우 (Trade = "buy")
           - 매도 (Trade = "")
        - 현재 구매 상태가 아닌 경우 (Trade = "")
           - 아무 행동도 하지 않는다 (Trade = "")

    2. 하단 밴드보다 종가가 낮은 경우
        - 현재 구매 상태인 경우 (Trade = "buy")
           - 아무 행동도 하지 않는다 (Trade = "buy")
        - 현재 구매 상태가 아닌 경우 (Trade = "")
           - 매수 (Trade = "buy")

    3. 하단 밴드와 상단 밴드 사이에 종가가 존재하는 경우
        - 현재 구매 상태인 경우 (Trade = "buy")
           - 아무 행동도 하지 않는다 (Trade = "buy")
        - 현재 구매 상태가 아닌 경우 (Trade = "")
           - 아무 행동도 하지 않는다 (Trade = "")

In [34]:
# price_df2의 인덱스 값을 출력
price_df2.index

Index(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
       '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15',
       ...
       '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-17',
       '2019-06-18', '2019-06-19', '2019-06-20', '2019-06-21', '2019-06-24'],
      dtype='object', name='Date', length=2384)

In [35]:
# 8. 구매내역 추가
# 반복문 
for i in price_df2.index :
    # 상단 밴드보다 종가가 높은 경우
    if price_df2.loc[i, "Adj Close"] > price_df2.loc[i, "Ub"] :
        # 현재 구매 상태인 경우(전 날의 Trade = "buy")
        if price_df2.shift(1).loc[i, "Trade"] == "buy" :
            # 매도
            price_df2.loc[i, "Trade"] = ""
        # 현재 구매 상태가 아닌 경우(전 날의 Trade = "")
        else : price_df2.loc[i, "Trade"] = ""
    # 하단 밴드보다 종가가 낮은 경우
    elif price_df2.loc[i, "Adj Close"] < price_df2.loc[i, "Lb"] : 
        # 현재 구매 상태인 경우(전 날의 Trade = "buy")
        if price_df2.shift(1).loc[i, "Trade"] == "buy" :
            # 현 상태를 유지
            price_df2.loc[i, "Trade"] = "buy"
        # 현재 구매 상태가 아닌 경우(전 날의 Trade = "")
        else : price_df2.loc[i, "Trade"] = "buy"
    # 하단 밴드와 상단 밴드 사이에 종가가 존재하는 경우 
    elif price_df2.loc[i, "Adj Close"] >= price_df2.loc[i, "Lb"] and price_df2.loc[i, "Adj Close"] <= price_df2.loc[i, "Ub"] :
        # 현재 구매 상태인 경우(전 날의 Trade = "buy")
        if price_df2.shift(1).loc[i, "Trade"] == "buy" : 
            # 현 상태를 유지
            price_df2.loc[i, "Trade"] = "buy"
        # 현재 구매 상태가 아닌 경우(전 날의 Trade = "")
        else :
            price_df2.loc[i, "Trade"] = ""

In [36]:
price_df2["Trade"].value_counts()

       1521
buy     863
Name: Trade, dtype: int64

In [42]:
# apply를 이용하여 구매 내역 추가
def bol(x) :
    close = x[0]
    ub = x[1]
    lb = x[2]
    trade = price_df2.shift(1)[(price_df2["Adj Close"] == close) & (price_df2["Ub"] == ub) & (price_df2["Lb"] == lb)]["Trade"].values
    if close > ub :
        if trade == "buy" :
            return ""
        else :
            return ""
    elif close < lb : 
        return "buy"
    else : 
        if trade == "buy" :
            return "buy"
        else : 
            return ""
        
price_df2[["Adj Close", "Ub", "Lb"]].apply(bol, axis = 1, raw = 1).value_counts()

       1521
buy     863
dtype: int64

## 수익율 계산
- 구매한 날의 종가
    - Trade 컬럼에서 전 행의 Trade가 "", 현재 행의 Trade가 "buy"인 날의 종가 = 구매 가격
- 판매한 날의 종가
    - 전 행의 Trade가 "buy"이고 현재 행의 Trade가 ""인 날의 종가 = 판매 가격
- 수익율 계산
    - (판매 가격 - 구매 가격) / 구매가격 + 1
- 구매 가격과 판매 가격 초기화
- 여러 개의 수익율 발생
- 누적 수익율
    - 수익율 누적으로 곱

In [ ]:
# 9. 손익 계산
rtn = 1.0
price_df2["Return"] = 1
# 구매 가격과 판매 가격 변수 생성
buy = 0.0
sell = 0.0

for i in price_df2.index :
    # 구매 가격 확인
    if price_df2.shift(1).loc[i, "Trade"] == "" and price_df2.loc[i, "Trade"] == "buy" :
        buy = price_df2.loc[i, "Adj Close"]
        print("구매 일 :", i, "구매 가격 :", buy)
    # 판매 가격 확인
    elif price_df2.shift(1).loc[i, "Trade"] == "buy" and price_df2.loc[i, "Trade"] == "" :
        sell = price_df2.loc[i, "Adj Close"]
        # 수익율 계산
        rtn = (sell - buy) / buy +1
        # 수익율을 Return 컬럼에 대입
        price_df2.loc[i, "Return"] = rtn
        print("판매일 :", i, "판매 가격 :", sell, "수익율 :", round(rtn, 4))
    
    # 구매 가격과 판매 가격을 초기화
    if price_df2.loc[i, "Trade"] == "" : 
        buy = 0.0
        sell = 0.0


In [44]:
price_df2.loc["2010-08-04"]

Adj Close    127.580002
Center          119.719
Ub           125.445332
Lb           113.992669
Trade                  
Return          0.98267
Name: 2010-08-04, dtype: object

In [72]:
# 누적 수익률


# 수익율 변수 생성
acc_rtn = 1.0

for i in price_df2.index :
    rtn = price_df2.loc[i, "Return"]
    acc_rtn *= rtn
    price_df2["Acc_rtn"] = acc_rtn

print("누적 수익율 :", round(acc_rtn, 4))

C:\Users\Kim Taemin\AppData\Local\Temp\ipykernel_24480\2466340064.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df2["Acc_rtn"] = acc_rtn


누적 수익율 : 3.1381


In [49]:
# 데이터프레임의 첫번째 인덱스 출력
buy = price_df2.iloc[0, 0]

In [50]:
# 데이터프레임의 마지막 인덱스 출력
sell = price_df2.iloc[-1, 0]

In [52]:
(sell - buy) / buy +1

14.249095911087197

# 볼린저 밴드를 함수화
- 1번 함수 생성
    - 매개변수는 데이터프레임. 기준이 되는 컬럼, 시작 시간
    - 데이터프레임의 결측치와 이상치를 제거
    - 기준이 되는 컬럼을 제외하고 나머지 컬럼을 삭제
    - 이동 평균선, 상단 밴드, 하단 밴드 파생 변수 생성
    - 시작 시간부터 마지막 시간까지 데이터 프레임을 필터링 한 뒤 데이터프레임을 리턴
- 2번 함수 생성
    - 매개변수 데이터프레임(1번 함수에서 만든 것을 받아 옴)
    - Trade 컬럼을 생성 -> ""
    - 구매내역을 Trade 컬럼에 대입 후 데이터프레임 리턴
- 3번 함수 생성
    - 매개변수 데이터프레임(2번 함수에서 만든 것을 받아 옴)
    - Return 컬럼을 생성 -> 1
    - 손익 계산을 하여 Return 컬럼에 대입
    - 누적 수익율 Acc_rtn 컬럼을 생성하고 누적 수익율 대입
    - 누적 수익율 리턴

In [53]:
# 1번 함수 생성
def first(_df, _col, _time) :
    # 결측치와 이상치를 제거
    result = _df[~_df.isin([np.nan, np.inf, -np.inf]).any(1)]
    # 기준이 되는 컬럼을 제외한 나머지 컬럼을 삭제
    result = result.loc[:, [_col]]
    # 이동 평균선, 상단 밴드, 하단 밴드 생성
    result["Center"] = result[_col].rolling(20).mean()
    result["Ub"] = result["Center"] + (2 * result[_col].rolling(20).std())
    result["Lb"] = result["Center"] - (2 * result[_col].rolling(20).std())

    # 시작 시간부터 마지막 데이터까지 필터링
    result = result.loc[_time :]

    # 결과를 리턴
    return result

In [68]:
df = pd.read_csv("../csv/AAPL.csv", index_col = "Date")

In [69]:
df = first(df, "Close", "2013-01-01")

C:\Users\Kim Taemin\AppData\Local\Temp\ipykernel_24480\1358551224.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  result = _df[~_df.isin([np.nan, np.inf, -np.inf]).any(1)]


In [57]:
# 2번 함수
# 거래 내역을 추가하는 함수
def second(_df) :
    # Trade 컬럼을 생성 -> 값은 ""
    result = _df
    result["Trade"] = ""
    # 기준이 되는 컬럼의 이름은 가변
    # 위치는 변하지 않기 때문에
    # 기준이 되는 컬럼은 첫번째
    col = result.columns[0]

    # Trade에 내역 추가
    for i in result.index :
        # 기준이 되는 컬럼이 상단 밴드보다 큰 경우
        if result.loc[i, col] > result.loc[i, "Ub"] :
            if result.shift(1).loc[i, "Trade"] == "buy" : 
                result.loc[i, "Trade"] = ""
            else : 
                result.loc[i, "Trade"] = ""
        # 기준이 되는 컬럼이 하단 밴드보다 작은 경우
        if result.loc[i, col] < result.loc[i, "Lb"] :
            if result.shift(1).loc[i, "Trade"] == "buy" : 
                result.loc[i, "Trade"] = "buy"
            else : 
                result.loc[i, "Trade"] = "buy"
        # 기준이 되는 컬럼이 하단 밴드와 상단 밴드 사이에 존재하는 경우
        if result.loc[i, col] >= result.loc[i, "Lb"] and result.loc[i, col] <= result.loc[i, "Ub"] :
            if result.shift(1).loc[i, "Trade"] == "buy" : 
                result.loc[i, "Trade"] = "buy"
            else : 
                result.loc[i, "Trade"] = ""
    return result

In [60]:
second(df)["Trade"].value_counts()

       933
buy    697
Name: Trade, dtype: int64

In [66]:
# 3번 함수
# 수익율을 계산하는 함수
def third(_df) : 
    result = _df
    # Return 컬럼을 생성 -> 값 1
    result["Return"] = 1
    # 기준이 되는 컬럼은 columns[0]
    col = result.columns[0]
    # 수익율, 구매 가격, 판매 가격 변수 생성
    rtn = 1.0
    buy = 0.0
    sell = 0.0
    
    # 수익율을 계산하는 반복문
    for i in result.index :
       # 구매 가격 확인
       if result.shift(1).loc[i, "Trade"] == "" and result.loc[i, "Trade"] == "buy" :
           buy = result.loc[i, col]
           print("구매 일 :", i, "구매 가격 :", buy)
           # 판매 가격 확인
       elif result.shift(1).loc[i, "Trade"] == "buy" and result.loc[i, "Trade"] == "" :
           sell = result.loc[i, col]
           # 수익율 계산
           rtn = (sell - buy) / buy +1
           # 수익율을 Return 컬럼에 대입
           result.loc[i, "Return"] = rtn
           print("판매일 :", i, "판매 가격 :", sell, "수익율 :", round(rtn, 4))
    
        # 구매 가격과 판매 가격을 초기화
       if result.loc[i, "Trade"] == "" : 
           buy = 0.0
           sell = 0.0
    # 누적 수익률


    # 수익율 변수 생성
    acc_rtn = 1.0

    for i in result.index :
        rtn = result.loc[i, "Return"]
        acc_rtn *= rtn
        result.loc[i, "Acc_rtn"] = acc_rtn

    print("누적 수익율 :", round(acc_rtn, 4))

    return result


In [ ]:
df = second(df)
third(df)

In [74]:
import bollinger as bb
import imp
imp.reload(bb)

In [76]:
df = pd.read_csv("../csv/BND.csv", index_col = "Date")
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-04-10,75.070000,75.260002,75.000000,75.239998,51.523693,35000
2007-04-11,75.160004,75.290001,75.029999,75.040001,51.386742,87700
2007-04-12,75.059998,75.080002,74.959999,75.029999,51.379879,78100
2007-04-13,75.040001,75.070000,74.849998,74.910004,51.297688,18000
2007-04-16,74.989998,74.989998,74.940002,74.980003,51.345627,52700


In [77]:
df = bb.first(df, "Adj Close", "2011-01-01")
df.head()

c:\Users\Kim Taemin\Documents\GitHub\python-class\230318\bollinger\__init__.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  result = _df[~_df.isin([np.nan, np.inf, -np.inf]).any(1)]


,Adj Close,Center,Ub,Lb
Date,,,,
2011-01-03,64.215347,64.013149,64.616070,63.410227
2011-01-04,64.223389,63.983159,64.463678,63.502641
2011-01-05,63.943043,63.962609,64.410380,63.514838
2011-01-06,63.999104,63.961561,64.408864,63.514259
2011-01-07,64.239395,63.969743,64.431591,63.507896


In [79]:
df = bb.second(df)
df[18:25]

,Adj Close,Center,Ub,Lb,Trade
Date,,,,,
2011-01-28,64.439621,64.253806,64.511981,63.995631,
2011-01-31,64.351532,64.256611,64.517890,63.995332,
2011-02-01,64.227013,64.257194,64.518137,63.996251,
2011-02-02,64.154724,64.253761,64.518358,63.989164,
2011-02-03,63.937851,64.253501,64.519389,63.987613,buy
2011-02-04,63.720966,64.239594,64.580125,63.899064,buy
2011-02-07,63.728996,64.214075,64.624081,63.804068,buy


In [81]:
df = bb.third(df)
df[18:25]

구매 일 : 2011-02-03 구매 가격 : 63.937851
판매일 : 2011-03-16 판매 가격 : 65.105652 수익율 : 1.0183
구매 일 : 2011-06-29 구매 가격 : 66.049957
판매일 : 2011-08-01 판매 가격 : 67.197464 수익율 : 1.0174
구매 일 : 2011-11-30 구매 가격 : 68.584297
판매일 : 2011-12-13 판매 가격 : 69.032143 수익율 : 1.0065
구매 일 : 2012-03-14 구매 가격 : 69.140411
판매일 : 2012-04-10 판매 가격 : 70.015167 수익율 : 1.0127
구매 일 : 2012-08-15 구매 가격 : 71.121567
판매일 : 2013-02-25 판매 가격 : 71.270378 수익율 : 1.0021
구매 일 : 2013-05-10 구매 가격 : 71.452927
판매일 : 2013-09-18 판매 가격 : 69.627502 수익율 : 0.9745
구매 일 : 2013-12-24 구매 가격 : 69.648918
판매일 : 2014-02-03 판매 가격 : 70.937912 수익율 : 1.0185
구매 일 : 2014-09-10 구매 가격 : 72.17923
판매일 : 2014-10-01 판매 가격 : 72.65773 수익율 : 1.0066
구매 일 : 2015-02-17 구매 가격 : 73.605263
판매일 : 2015-03-18 판매 가격 : 74.53598 수익율 : 1.0126
구매 일 : 2015-04-28 구매 가격 : 74.316902
판매일 : 2015-07-08 판매 가격 : 73.50975 수익율 : 0.9891
구매 일 : 2015-10-29 구매 가격 : 73.890236
판매일 : 2015-12-01 판매 가격 : 74.04142 수익율 : 1.002
구매 일 : 2016-09-09 구매 가격 : 77.30217
판매일 : 2017-01-05 판매 가격 : 75.613007 수익율 : 0.9781

,Adj Close,Center,Ub,Lb,Trade,Return,Acc_rtn
Date,,,,,,,
2011-01-28,64.439621,64.253806,64.511981,63.995631,,1.0,1.0
2011-01-31,64.351532,64.256611,64.517890,63.995332,,1.0,1.0
2011-02-01,64.227013,64.257194,64.518137,63.996251,,1.0,1.0
2011-02-02,64.154724,64.253761,64.518358,63.989164,,1.0,1.0
2011-02-03,63.937851,64.253501,64.519389,63.987613,buy,1.0,1.0
2011-02-04,63.720966,64.239594,64.580125,63.899064,buy,1.0,1.0
2011-02-07,63.728996,64.214075,64.624081,63.804068,buy,1.0,1.0
